## This is the awesome code that helps me make my thesis
 1. Goes to every web page from the congressional reords and downloads the pdf
 2. Storages the pdf
 3. Scrapes the pdf, god knows how
 4. Deletes the pdf
 5. Stores the text data as a txt
 ------------------------------------------------------------------------------------------------------
 Eventually I have to make a more comprehensive dataset that stores information about every Senator, pottentially with entity dissambiguation
 6. Defines the columns of this awesome df as "Date", "Name of Senator", "Discourse"


In [173]:
!pip install pytesseract
!python3 -m pip install selenium

Defaulting to user installation because normal site-packages is not writeable
^C

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
ERROR: Operation cancelled by user
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [179]:
################ LOAD YOUR PACKAGES #########################
import pandas as pd # To manage data frames

import time
from time import sleep
from datetime import date, timedelta 
from datetime import datetime

import pytesseract # To extract text with OCR
import os # To manage file paths
from os import listdir
from os.path import isfile, join
from os import path
from pathlib import Path

from selenium import webdriver # To navigate to each page and find the url to download
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By # To locate elements
from selenium.common.exceptions import NoSuchElementException  # Import the exception
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import urllib.request # To dowload the pdf file
import csv

In [204]:
################ DEFINE YOUR PATH #########################
root = Path('/Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis')
root_out = root / 'input'

download_dir = root_out / 'Downloaded_CR'

In [168]:
################ PARAMETERS TO SET #########################

#Specify the time frame for which we want to download data.
#All dates within this time frame *which have not been downloaded yet* will be downloaded.
#The dates that have already been stores are stored in a CSV file ("dates_info.txt")

start_year=1979
start_month=1
start_day=21
end_year=1979
end_month=12
end_day=31

##Creating date list
#We initialize a  date list which will contain all of the dates for which we want to scrap data.
#To initialize this list, we need to load information contained in log files.

date_list=[]
d1=date(start_year,start_month,start_day)
d2=date(end_year,end_month,end_day)
delta = d2 - d1

#We build the list of dates to gather: dates which are in the time frame and which are not in one of the log files
for i in range(delta.days + 1):
    d=d1 + timedelta(i)
    d=str(d)
    elements=d.split("-")
    datum=elements[1]+"-"+elements[2]+"-"+elements[0]
    if not datum in already_downloaded:
       # if not datum in too_long_list:
            if not datum in empty_list:
                date_list.append(datum)

# Define Selenium options

In [171]:
#We initialize paramters for the scraper
# Instead of the path created ith os i just place the path as a string, because the os element is a PosixPath, which is not JSON serializable
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2,"download.default_directory" : '/Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/input/Downloaded_CR' }
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.set_capability("pageLoadStrategy", "eager") 

# These two should help me have a faster execution and not kill my computer
# But looks like the data I want relys on JavaScript to load the content, so... I need to open them
# chrome_options.add_argument("--headless")  # Run in headless mode
# chrome_options.add_argument("--disable-images")  # Disable images

# Initialize Selenium and collect the urls!!

In [175]:

i = 0
#We loop over dates which need to be retrieved.
dates_info = []

driver = webdriver.Chrome(options=chrome_options)

while i < len(date_list):
    print(i)
    chamber = "Senate"
    problematic_date = 0  # Dummy to assess whether the date was problematic
    
    #Dummy to assess whether the date was problematic
    problematic_date = 0

    #We launch the scraper
    driver.set_page_load_timeout(300)
   
    #We create the url for which we want to retrive data
    # Split and rearrange the components to '1960/01/07'
    date_parts = date_list[i].split('-')  # Split into ['01', '07', '1960']
    formatted_date = f"{date_parts[2]}/{date_parts[0]}/{date_parts[1]}"
    year_interest=int(date_list[i].split("-")[2])
    period=""
    if year_interest%2==1:
        period=str(year_interest+1)
    else:
        period=str(year_interest)
    driver.delete_all_cookies()
    url="https://www.congress.gov/bound-congressional-record/"+formatted_date+"/senate-section"
    #We go to the url of interest
    try:
        # We go to the url of interest
        driver.get(url)
        html = driver.page_source

        try:
            # Locate the div with class "cdg-summary-wrapper"
            summary_div = driver.find_element(By.CLASS_NAME, "cdg-summary-wrapper")
            # Find the <a> tag inside the div
            link = summary_div.find_element(By.TAG_NAME, "a")
            # Get the href attribute
            href = link.get_attribute("href")
            # Get the text attribute
            description = link.get_attribute("text")
            description = description.strip()
            description = ' '.join(description.split())

        except NoSuchElementException:
            # Handle the case where the element does not exist
            href = None
            description = "This date did not hold a Congressional Debate in the Senate floor"
            description = description.replace('\n', ' ')

    except TimeoutException:
        # Handle the case where the page load time exceeds 5 minutes
        href = None
        description = "Did not load in proper time"
           
    print(url)
    dates_info.append({
        "date": date_list[i],
        "url": url,
        "href": href,
        "description": description,
        "chamber": chamber
    })
    
    # Increment i to move to the next date in the list
    i += 1
    # Close the driver after processing each date
driver.quit()
    
# Convert the list to a DataFrame
dates_infodf = pd.DataFrame(dates_info)

0
https://www.congress.gov/bound-congressional-record/1979/01/21/senate-section
1
https://www.congress.gov/bound-congressional-record/1979/01/22/senate-section
2
https://www.congress.gov/bound-congressional-record/1979/01/23/senate-section
3
https://www.congress.gov/bound-congressional-record/1979/01/24/senate-section
4
https://www.congress.gov/bound-congressional-record/1979/01/25/senate-section
5
https://www.congress.gov/bound-congressional-record/1979/01/26/senate-section
6
https://www.congress.gov/bound-congressional-record/1979/01/27/senate-section
7
https://www.congress.gov/bound-congressional-record/1979/01/28/senate-section
8
https://www.congress.gov/bound-congressional-record/1979/01/29/senate-section
9
https://www.congress.gov/bound-congressional-record/1979/01/30/senate-section
10
https://www.congress.gov/bound-congressional-record/1979/01/31/senate-section
11
https://www.congress.gov/bound-congressional-record/1979/02/01/senate-section
12
https://www.congress.gov/bound-cong

In [176]:
dates_infodf = pd.DataFrame(dates_info)

In [183]:
dates_info_pth = os.path.join(root_out, 'dates_info.csv') 

# Check if the CSV file already exists
if os.path.exists(dates_info_pth):
    # Read the existing CSV file
    existing_data = pd.read_csv(dates_info_pth)
    # Append the new data to the existing data
    updated_data = pd.concat([existing_data, dates_infodf], ignore_index=True)
else:
    # If the file doesn't exist, use the new data as is
    updated_data = dates_infodf

# Save the updated data back to the CSV file
updated_data.to_csv(dates_info_pth, index=False)

In [211]:
dates_info_pth = os.path.join(root_out, 'dates_info.csv') 

In [208]:
import requests
import shutil
import time
from pathlib import Path
from datetime import datetime
import csv

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

def download_pdf(url, filename, max_retries=3):
    for attempt in range(max_retries):
        try:
            with requests.get(url, headers=headers, stream=True) as response:
                response.raise_for_status()
                with open(filename, 'wb') as file:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:  # Filter out keep-alive new chunks
                            file.write(chunk)
                print(f"PDF downloaded successfully as {filename}.")
                break  # Exit the loop if successful
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                print(f"Failed to download PDF after {max_retries} attempts: {url}")

def standardize_date(date_str):
    for fmt in ("%m-%d-%Y", "%m/%d/%Y", "%d-%m-%Y", "%d/%m/%Y"):
        try:
            date_obj = datetime.strptime(date_str, fmt)
            return date_obj.strftime("%d/%m/%Y")
        except ValueError:
            continue
    raise ValueError(f"Date format not recognized: {date_str}")

In [212]:

with open(dates_info_pth, mode='r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';')
    print("Column names in CSV:", reader.fieldnames)  # Debugging: Print column names
    
    for row in reader:
        if not row:  # Skip empty rows
            continue
        
        href = row.get('href')  # Use .get() to avoid KeyError
        date_str = row.get('date')  # Use .get() to avoid KeyError
        
        if href and date_str:
            try:
                standardized_date = standardize_date(date_str)
                pdf_filename = download_dir / f"{standardized_date.replace('/', '-')}.pdf"
                download_pdf(href, pdf_filename)
            except ValueError as e:
                print(f"Skipping row {reader.line_num}: {e}")
        else:
            print(f"Skipping row {reader.line_num}: Missing 'href' or 'date'")

Column names in CSV: ['date', 'url', 'href', 'description', 'chamber']
Skipping row 2: Missing 'href' or 'date'
PDF downloaded successfully as /Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/input/Downloaded_CR/15-01-1979.pdf.
Skipping row 4: Missing 'href' or 'date'
Skipping row 5: Missing 'href' or 'date'
PDF downloaded successfully as /Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/input/Downloaded_CR/18-01-1979.pdf.
PDF downloaded successfully as /Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/input/Downloaded_CR/19-01-1979.pdf.
Skipping row 8: Missing 'href' or 'date'
Skipping row 9: Missing 'href' or 'date'
Skipping row 10: Missing 'href' or 'date'
Skipping row 11: Missing 'href' or 'date'
PDF downloaded successfully as /Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/input/Downloaded_CR/24-01-1979.pdf.
PDF downloaded successfully as /User